In [1]:
# nvidia-smi
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:

import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from PIL import Image
from concurrent.futures import ThreadPoolExecutor
import torch
import torch.optim as optim
import torch.nn.functional as F

from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv

from utils.create_data_for_close import get_graph_from_svg

/home/ivy8792/anaconda3/envs/graph/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# load images
svg_folder = './datasets/svg'
png_folder = './datasets/png'
imgs = []
png = []
dataset = []

for root, folders, files in os.walk(svg_folder):
    for file in files:
        if file.split('.')[1] != 'svg': continue
        if 'checkpoint' in file: continue
        
        file_path = os.path.join(svg_folder, file)
        imgs.append(file_path)
        
        file_path = os.path.join(png_folder, file.replace('svg', 'png'))
        png.append(file_path)

In [5]:
# 6m 14s
for i, file_path in enumerate(tqdm(imgs)):
    # try:
    #     dataset.append(get_graph_from_svg(file_path))
    # except:
    #     print(file_path)
        
    file_path = "./datasets/svg/042-tree.svg"
    get_graph_from_svg(file_path)
    break

  0%|          | 0/5440 [00:00<?, ?it/s]


ValueError: setting an array element with a sequence. The requested array would exceed the maximum number of dimension of 1.

In [22]:
print(dataset[0])

Data(x=[5, 5], edge_index=[2, 26], edge_attr=[26, 1], y=[5, 3])


In [10]:
# hyperparameters
torch.manual_seed(16)

batch_size = 16
num_features = 5  # (R, G, B, x, y)
num_output = 3  # (R, G, B)
num_epoch = 500

_train = int(len(dataset) * 0.85)
_val = _train + int(len(dataset) * 0.1)
_test = len(dataset) - _val

# create dataloader
train_set, val_set, test_set = dataset[:_train], dataset[_train:_val], dataset[_val:]
train_svg, val_svg, test_svg = imgs[:_train], imgs[_train:_val], imgs[_val:]
train_png, val_png, test_png = png[:_train], png[_train:_val], png[_val:]

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

print(f"Training Data: {len(train_set)}\nValidation Data: {len(val_set)}\nTesting Data: {len(test_set)}")

Training Data: 4624
Validation Data: 544
Testing Data: 272


## GCN model

In [11]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(num_features, 16)
        self.conv2 = GCNConv(16, num_output)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)

        return x

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.MSELoss(reduction='mean')

In [13]:
# training
train_losses = []
val_losses = []
best_loss = float('inf')

for epoch in range(2):  # num_epoch
    train_loss = 0
    val_loss = 0
    
    model.train()
    for i, data in enumerate(tqdm(train_loader)):
        data = data.to(device)
        
        optimizer.zero_grad()
        out = model(data)  # out.cpu().detach().numpy().shape = (num_node, 3)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        
    model.eval()
    for i, data in enumerate(tqdm(val_loader)):
        data = data.to(device)
        out = model(data)
        loss = criterion(out, data.y)
        val_loss += loss.item()
    
    train_avg = train_loss / len(train_loader)
    val_avg = val_loss / len(val_loader)
    train_losses.append(loss.item())
    val_losses.append(loss.item())
    
    print(f'Epoch {epoch}\tTraining Loss: {train_avg}\tValidation Loss: {val_avg}')
    
    if val_avg < best_loss:
        print(f'Validation Loss Decreased({best_loss:.6f}--->{val_avg:.6f})\tSaving The Model')
        best_loss = val_avg
        torch.save(model.state_dict(), 'best_checkpoint.pth')

100%|██████████| 34/34 [00:00<00:00, 455.29it/s]


Epoch 0	Training Loss: 0.5904875048097855	Validation Loss: 0.39589134384604063
Validation Loss Decreased(inf--->0.395891)	Saving The Model


100%|██████████| 34/34 [00:00<00:00, 466.53it/s]

Epoch 1	Training Loss: 0.42965951788796686	Validation Loss: 0.4004649414735682


In [ ]:
# plot losses
_x = list(range(num_epoch))
plt.plot(_x, train_losses, label='Training Loss')
plt.plot(_x, val_losses, label='Validation Loss')
 
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# testing
model.load_state_dict(torch.load('./best_checkpoint.pth'))
model.eval()

test_loss = 0
for i, data in enumerate(tqdm(test_loader)):
    data = data.to(device)
    out = model(data)
    loss = criterion(out, data.y)
    test_loss += loss.item()
    
print(f'Testing Loss: {test_loss / len(test_loader)}')

In [ ]:
# visualize testing results
vis_loader = DataLoader(test_set, batch_size=1, shuffle=False)

model.load_state_dict(torch.load('./best_checkpoint.pth'))
model.eval()

for data, png in zip(vis_loader, test_png):
    data = data.to(device)
    out = model(data)
    
    pos = data.x.cpu().detach().numpy()[:,3:]
    rgb = out.cpu().detach().numpy()
    
    # get pos and hex color
    n_rgb = rgb * 255
    cc = []
    for r, g, b in n_rgb:
        cc.append("#" + ('{:X}{:X}{:X}').format(int(r), int(g), int(b)))
    
    # plot output color
    plt.subplot(121)
    img = Image.open(png)
    plt.axis("off")
    plt.imshow(img)
    
    plt.subplot(122)
    # plt.axis([0, 25, 33, -8])
    for (x, y), c in zip(pos, cc):
        plt.scatter(x, y, c=c)
    plt.axis("off")
    plt.show()
    plt.close()
    break